In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-p10/Feature_engineering.py
/kaggle/input/2021-ml-p10/activity_labels.txt
/kaggle/input/2021-ml-p10/features_info.txt
/kaggle/input/2021-ml-p10/data_loader.py
/kaggle/input/2021-ml-p10/features.txt
/kaggle/input/2021-ml-p10/sample_submit.csv
/kaggle/input/2021-ml-p10/RawData/acc_exp28_user14.txt
/kaggle/input/2021-ml-p10/RawData/acc_exp45_user22.txt
/kaggle/input/2021-ml-p10/RawData/acc_exp15_user08.txt
/kaggle/input/2021-ml-p10/RawData/acc_exp05_user03.txt
/kaggle/input/2021-ml-p10/RawData/gyro_exp13_user07.txt
/kaggle/input/2021-ml-p10/RawData/acc_exp43_user21.txt
/kaggle/input/2021-ml-p10/RawData/acc_exp11_user06.txt
/kaggle/input/2021-ml-p10/RawData/gyro_exp26_user13.txt
/kaggle/input/2021-ml-p10/RawData/acc_exp39_user19.txt
/kaggle/input/2021-ml-p10/RawData/gyro_exp05_user03.txt
/kaggle/input/2021-ml-p10/RawData/label_train.txt
/kaggle/input/2021-ml-p10/RawData/acc_exp13_user07.txt
/kaggle/input/2021-ml-p10/RawData/gyro_exp54_user27.txt
/kaggle/input/2021-ml-p

# [센서데이터를 이용한 행동분류](https://www.kaggle.com/c/2021-ml-p10)
> 기계학습 9주차 실습 - DecisionTree

- https://www.kaggle.com/c/2021-ml-p10/discussion/235176

In [2]:
# Importing numpy 
import numpy as np
# Importing Scipy 
import scipy as sp
# Importing Pandas Library 
import pandas as pd
# import glob function to scrap files path
from glob import glob
# import display() for better visualitions of DataFrames and arrays
from IPython.display import display
# import pyplot for plotting
import matplotlib.pyplot as plt
plt.style.use('bmh') # for better plots
import tqdm

# import data_loader for data loading
from data_loader import import_raw_signals, import_labels_file,normalize5,normalize2

## Raw 데이터 불러오기
해당 실습문제에서는 데이터셋 로더를 제공합니다. 데이터를 다운받으시면 dataloader.py를 확인하실 수 있습니다. 해당 파이썬 코드에 정의되어 있는 'importrawsignals', 'importlabels_file' 함수를 통해서 Raw 데이터를 불러오시면 됩니다. 이때 캐글 노트북으로 진행하시는 분들은 경로 설정이 중요한데 다음과 같이 설정하시면 됩니다.

### Data Tree

```
+--RawData
|  +--acc_exp01_user01.txt
|  +--...(61개)
|  +--gyro_exp01_user01.txt
|  +--...(61개)
|  +--label_train.txt
|  +--label_test.txt

```

In [3]:
Raw_data_paths = sorted(glob("/kaggle/input/2021-ml-p10/RawData/*"))
Raw_acc_paths=Raw_data_paths[0:61]
Raw_gyro_paths=Raw_data_paths[61:122]

print (("RawData folder contains in total {:d} file ").format(len(Raw_data_paths)))
print (("The first {:d} are Acceleration files:").format(len(Raw_acc_paths)))
print (("The second {:d} are Gyroscope files:").format(len(Raw_gyro_paths)))
print ("The last file is a labels file")
print ("test labels file path is:",Raw_data_paths[122])
print ("train labels file path is:",Raw_data_paths[123])

RawData folder contains in total 124 file 
The first 61 are Acceleration files:
The second 61 are Gyroscope files:
The last file is a labels file
test labels file path is: /kaggle/input/2021-ml-p10/RawData/label_test.txt
train labels file path is: /kaggle/input/2021-ml-p10/RawData/label_train.txt


In [4]:
Raw_data_paths[0]

'/kaggle/input/2021-ml-p10/RawData/acc_exp01_user01.txt'

In [5]:
Raw_data_paths[0][-16:-4]

'exp01_user01'

In [6]:
raw_dic={}
raw_acc_columns=['acc_X','acc_Y','acc_Z']
raw_gyro_columns=['gyro_X','gyro_Y','gyro_Z']
for path_index in range(0,61):
        key= Raw_data_paths[path_index][-16:-4]
        raw_acc_data_frame=import_raw_signals(Raw_data_paths[path_index],raw_acc_columns)
        raw_gyro_data_frame=import_raw_signals(Raw_data_paths[path_index+61],raw_gyro_columns)
        raw_signals_data_frame=pd.concat([raw_acc_data_frame, raw_gyro_data_frame], axis=1)
        raw_dic[key]=raw_signals_data_frame

In [7]:
print('raw_dic contains %d DataFrame' % len(raw_dic))
display(raw_dic['exp01_user01'].head(3))

raw_dic contains 61 DataFrame


,acc_X,acc_Y,acc_Z,gyro_X,gyro_Y,gyro_Z
0,0.918056,-0.112500,0.509722,-0.054978,-0.069639,-0.030849
1,0.911111,-0.093056,0.537500,-0.012523,0.019242,-0.038485
2,0.881944,-0.086111,0.513889,-0.023518,0.276417,0.006414


In [8]:
len(raw_dic['exp01_user01'])

20598

In [9]:
raw_dic['exp01_user01'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20598 entries, 0 to 20597
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   acc_X   20598 non-null  float64
 1   acc_Y   20598 non-null  float64
 2   acc_Z   20598 non-null  float64
 3   gyro_X  20598 non-null  float64
 4   gyro_Y  20598 non-null  float64
 5   gyro_Z  20598 non-null  float64
dtypes: float64(6)
memory usage: 965.7 KB


In [10]:
len(raw_dic['exp02_user01'])

19286

In [11]:
len(raw_dic['exp03_user02'])

18026

In [12]:
len(raw_dic['exp04_user02'])

16565

In [13]:
raw_dic.keys()

dict_keys(['exp01_user01', 'exp02_user01', 'exp03_user02', 'exp04_user02', 'exp05_user03', 'exp06_user03', 'exp07_user04', 'exp08_user04', 'exp09_user05', 'exp10_user05', 'exp11_user06', 'exp12_user06', 'exp13_user07', 'exp14_user07', 'exp15_user08', 'exp16_user08', 'exp17_user09', 'exp18_user09', 'exp19_user10', 'exp20_user10', 'exp21_user10', 'exp22_user11', 'exp23_user11', 'exp24_user12', 'exp25_user12', 'exp26_user13', 'exp27_user13', 'exp28_user14', 'exp29_user14', 'exp30_user15', 'exp31_user15', 'exp32_user16', 'exp33_user16', 'exp34_user17', 'exp35_user17', 'exp36_user18', 'exp37_user18', 'exp38_user19', 'exp39_user19', 'exp40_user20', 'exp41_user20', 'exp42_user21', 'exp43_user21', 'exp44_user22', 'exp45_user22', 'exp46_user23', 'exp47_user23', 'exp48_user24', 'exp49_user24', 'exp50_user25', 'exp51_user25', 'exp52_user26', 'exp53_user26', 'exp54_user27', 'exp55_user27', 'exp56_user28', 'exp57_user28', 'exp58_user29', 'exp59_user29', 'exp60_user30', 'exp61_user30'])

In [14]:
len(raw_dic.keys())

61

## Label 불러오기
train, test 라벨을 불러오기 위한 방법은 다음과 같습니다.

Label 정보
```
train
experiment_number_ID : 실험 ID
user_number_ID : 유저 ID
activity_number_ID : 활동 ID
Label_start_point : Raw 데이터에서 행동이 시작하는 지점(시간)
Label_end_point : RAW 데이터에서 행동이 끝나는 지점(시간)

test
experiment_number_ID : 실험 ID
user_number_ID : 유저 ID
Label_start_point : Raw 데이터에서 행동이 시작하는 지점(시간)
Label_end_point : RAW 데이터에서 행동이 끝나는 지점(시간)
```

In [15]:
train_raw_labels_columns=['experiment_number_ID','user_number_ID','activity_number_ID','Label_start_point','Label_end_point']
test_raw_labels_columns=['experiment_number_ID','user_number_ID','Label_start_point','Label_end_point']

test_labels_path=Raw_data_paths[122]
train_labels_path=Raw_data_paths[123]

train_Labels_Data_Frame=import_labels_file(train_labels_path,train_raw_labels_columns)
test_Labels_Data_Frame=import_labels_file(test_labels_path,test_raw_labels_columns)

In [16]:
print ("The first 3 rows of  train_Labels_Data_Frame:" )
display(train_Labels_Data_Frame.head(3))
print(train_Labels_Data_Frame.shape)
display(test_Labels_Data_Frame.head(3))
print(test_Labels_Data_Frame.shape)

The first 3 rows of  train_Labels_Data_Frame:


,experiment_number_ID,user_number_ID,activity_number_ID,Label_start_point,Label_end_point
0,1,1,5,250,1232
1,1,1,4,1393,2194
2,1,1,5,2360,3374


(598, 5)


,experiment_number_ID,user_number_ID,Label_start_point,Label_end_point
0,3,2,298,1398
1,3,2,1686,2627
2,3,2,2770,3904


(258, 4)


## 데이터 전처리

해당 데이터셋을 제작한 저자는 논문을 통해 Raw 데이터 전처리 방법에 대해서 자세히 나타냈습니다. 먼저 RAW 데이터는 3축의 자이로 센서와 3축의 가속도계 센서를 통해 측정된 값들을 시간순서로 기록했습니다. 가장먼저 RAW 데이터에는 많은 노이즈가 있기 때문에 해당 노이즈에 대한 필터를 통해 처리를 진행해야 합니다. 논문에서 저자는 노이즈를 제거하기 위해 median filter와 3차 low pass Butterworth filter를 사용했다고 이야기합니다. 이때 Butter worth filter에 coner frequency는 20Hz로 설정했다고 합니다. 또한 가속도계 센서의 값을 body 가속도와 gravity 가속도 신호로 분리하였는데, 여기에도 Butter worth filter가 사용됐으며 이때 coner frequency는 0.3Hz로 설정했다고 합니다.

### Median Filter

In [17]:
from scipy.signal import medfilt

def median(signal):
    array=np.array(signal)   
    med_filtered=sp.signal.medfilt(array, kernel_size=3)
    return  med_filtered

### Butterworth Filter를 통한 성분분해

In [18]:
from scipy.fftpack import fft  
from scipy.fftpack import fftfreq
from scipy.fftpack import ifft
import math 

sampling_freq = 50
nyq=sampling_freq/float(2)
freq1 = 0.3
freq2 = 20

# Function name: components_selection_one_signal

# Inputs: t_signal:1D numpy array (time domain signal); 

# Outputs: (total_component,t_DC_component , t_body_component, t_noise) 
#           type(1D array,1D array, 1D array)

# cases to discuss: if the t_signal is an acceleration signal then the t_DC_component is the gravity component [Grav_acc]
#                   if the t_signal is a gyro signal then the t_DC_component is not useful
# t_noise component is not useful
# if the t_signal is an acceleration signal then the t_body_component is the body's acceleration component [Body_acc]
# if the t_signal is a gyro signal then the t_body_component is the body's angular velocity component [Body_gyro]

def components_selection_one_signal(t_signal,freq1,freq2):
    t_signal=np.array(t_signal)
    t_signal_length=len(t_signal)
    f_signal=fft(t_signal)
    freqs=np.array(sp.fftpack.fftfreq(t_signal_length, d=1/float(sampling_freq)))# frequency values between [-25hz:+25hz]
    
    # DC_component: f_signal values having freq between [-0.3 hz to 0 hz] and from [0 hz to 0.3hz] 
    #                                                             (-0.3 and 0.3 are included)
    
    # noise components: f_signal values having freq between [-25 hz to 20 hz[ and from ] 20 hz to 25 hz] 
    #                                                               (-25 and 25 hz inculded 20hz and -20hz not included)
    
    # selecting body_component: f_signal values having freq between [-20 hz to -0.3 hz] and from [0.3 hz to 20 hz] 
    #                                                               (-0.3 and 0.3 not included , -20hz and 20 hz included)
    
    
    f_DC_signal=[] # DC_component in freq domain
    f_body_signal=[] # body component in freq domain numpy.append(a, a[0])
    f_noise_signal=[] # noise in freq domain
    
    for i in range(len(freqs)):# iterate over all available frequencies
        
        # selecting the frequency value
        freq=freqs[i]
        
        # selecting the f_signal value associated to freq
        value= f_signal[i]
        
        # Selecting DC_component values 
        if abs(freq)>0.3:# testing if freq is outside DC_component frequency ranges
            f_DC_signal.append(float(0)) # add 0 to  the  list if it was the case (the value should not be added)                                       
        else: # if freq is inside DC_component frequency ranges 
            f_DC_signal.append(value) # add f_signal value to f_DC_signal list
    
        # Selecting noise component values 
        if (abs(freq)<=20):# testing if freq is outside noise frequency ranges 
            f_noise_signal.append(float(0)) # # add 0 to  f_noise_signal list if it was the case 
        else:# if freq is inside noise frequency ranges 
            f_noise_signal.append(value) # add f_signal value to f_noise_signal

        # Selecting body_component values 
        if (abs(freq)<=0.3 or abs(freq)>20):# testing if freq is outside Body_component frequency ranges
            f_body_signal.append(float(0))# add 0 to  f_body_signal list
        else:# if freq is inside Body_component frequency ranges
            f_body_signal.append(value) # add f_signal value to f_body_signal list
    
    ################### Inverse the transformation of signals in freq domain ########################
    # applying the inverse fft(ifft) to signals in freq domain and put them in float format
    t_DC_component= ifft(np.array(f_DC_signal)).real
    t_body_component= ifft(np.array(f_body_signal)).real
    t_noise=ifft(np.array(f_noise_signal)).real
    
    total_component=t_signal-t_noise # extracting the total component(filtered from noise) 
                                     #  by substracting noise from t_signal (the original signal).
    
    # return outputs mentioned earlier
    return (total_component,t_DC_component,t_body_component,t_noise) 

## 피처모델링


### 1. 유클리드 Norm을 통한 가속도,자이로 센서 크기(Magnitude) 계산

In [19]:
import math
def mag_3_signals(x,y,z): # Euclidian magnitude
    return [math.sqrt((x[i]**2+y[i]**2+z[i]**2)) for i in range(len(x))]

### 2. 미분을 통한 변화량(Jerk) 계산

In [20]:
dt=0.02 # dt=1/50=0.02s time duration between two rows
# Input: 1D array with lenght=N (N:unknown)
# Output: 1D array with lenght=N-1
def jerk_one_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [21]:
time_sig_dic={}
raw_dic_keys=sorted(raw_dic.keys())

for key in tqdm.tqdm(raw_dic_keys):
    raw_df=raw_dic[key]
    time_sig_df=pd.DataFrame()
    
    for column in raw_df.columns:
        t_signal=np.array(raw_df[column])
        med_filtred=median(t_signal)
        
        if 'acc' in column:
            _,grav_acc,body_acc,_=components_selection_one_signal(med_filtred,freq1,freq2)
            body_acc_jerk=jerk_one_signal(body_acc)
            time_sig_df['t_body_'+column]=body_acc[:-1]
            time_sig_df['t_grav_'+column]= grav_acc[:-1]
            time_sig_df['t_body_acc_jerk_'+column[-1]]=body_acc_jerk
        elif 'gyro' in column:
            _,_,body_gyro,_=components_selection_one_signal(med_filtred,freq1,freq2)
            body_gyro_jerk=jerk_one_signal(body_gyro)
            time_sig_df['t_body_gyro_'+column[-1]]=body_gyro[:-1]
            time_sig_df['t_body_gyro_jerk_'+column[-1]]=body_gyro_jerk
            
    new_columns_ordered=['t_body_acc_X','t_body_acc_Y','t_body_acc_Z',
                      't_grav_acc_X','t_grav_acc_Y','t_grav_acc_Z',
                      't_body_acc_jerk_X','t_body_acc_jerk_Y','t_body_acc_jerk_Z',
                      't_body_gyro_X','t_body_gyro_Y','t_body_gyro_Z',
                      't_body_gyro_jerk_X','t_body_gyro_jerk_Y','t_body_gyro_jerk_Z']
        
    ordered_time_sig_df=pd.DataFrame()
        
    for col in new_columns_ordered:
        ordered_time_sig_df[col]=time_sig_df[col]
        
    for i in range(0,15,3):
        mag_col_name=new_columns_ordered[i][:-1]+'mag'
        col0=np.array(ordered_time_sig_df[new_columns_ordered[i]]) # copy X_component
        col1=ordered_time_sig_df[new_columns_ordered[i+1]] # copy Y_component
        col2=ordered_time_sig_df[new_columns_ordered[i+2]] # copy Z_component
        mag_signal=mag_3_signals(col0,col1,col2)
        ordered_time_sig_df[mag_col_name]=mag_signal
        
    time_sig_dic[key]=ordered_time_sig_df

100%|██████████| 61/61 [01:58<00:00,  1.95s/it]


In [22]:
display(time_sig_dic['exp01_user01'].shape)
display(time_sig_dic['exp01_user01'].describe())
time_sig_dic['exp01_user01'].head(3)

(20597, 20)

,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_gyro_X,t_body_gyro_Y,t_body_gyro_Z,t_body_gyro_jerk_X,t_body_gyro_jerk_Y,t_body_gyro_jerk_Z,t_body_acc_mag,t_grav_acc_mag,t_body_acc_jerk_mag,t_body_gyro_mag,t_body_gyro_jerk_mag
count,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,2.059700e+04,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000
mean,0.000018,-0.000014,-0.000009,0.880358,-0.101501,0.097309,-0.001833,0.001183,0.000849,3.259619e-07,0.000008,0.000010,0.000046,0.000095,0.000362,0.165296,1.025479,3.404274,0.489705,9.439402
std,0.167505,0.123847,0.106477,0.299187,0.314444,0.262835,4.076298,3.237138,2.298076,3.907032e-01,0.497555,0.265791,7.641439,13.033311,5.724163,0.165557,0.025535,4.559251,0.480657,13.111758
min,-0.698870,-0.993486,-0.671478,-0.231975,-0.367943,-0.161622,-38.735217,-33.718078,-24.262993,-3.141414e+00,-2.694787,-4.050512,-90.662943,-158.457438,-89.662267,0.000864,0.768925,0.006144,0.001641,0.017986
25%,-0.044563,-0.024197,-0.042269,0.968003,-0.275034,-0.050273,-0.481136,-0.422376,-0.347403,-1.189116e-01,-0.139441,-0.059319,-1.100310,-1.705868,-0.832493,0.025437,1.021875,0.272200,0.087023,0.506674
50%,-0.001207,0.002857,-0.003154,0.997735,-0.236751,-0.025435,0.003867,0.003613,0.003452,3.737832e-03,-0.002389,0.003384,0.003147,-0.005965,-0.002999,0.114319,1.031724,1.581932,0.348679,4.510858
75%,0.020914,0.051177,0.026000,1.007358,-0.123642,0.090625,0.568717,0.505230,0.381417,1.546320e-01,0.101817,0.088021,1.159630,1.392834,0.799237,0.262351,1.038524,4.837894,0.758609,13.308692
max,1.031485,0.473201,0.493350,1.066684,0.832075,1.015052,34.647970,34.482683,35.302951,2.698454e+00,4.530312,2.260099,95.696959,151.910819,60.809206,1.180706,1.115731,54.209353,6.249451,160.671661


,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_gyro_X,t_body_gyro_Y,t_body_gyro_Z,t_body_gyro_jerk_X,t_body_gyro_jerk_Y,t_body_gyro_jerk_Z,t_body_acc_mag,t_grav_acc_mag,t_body_acc_jerk_mag,t_body_gyro_mag,t_body_gyro_jerk_mag
0,0.382899,-0.206562,-0.169117,0.431759,0.175976,0.720859,8.404087,-5.538872,-3.526643,-0.025624,-0.210093,-0.361419,0.552413,3.197618,-0.792801,0.466776,0.858500,10.665130,0.418831,3.340427
1,0.550980,-0.317339,-0.239649,0.443087,0.169277,0.715098,-9.140781,5.486163,3.704421,-0.014575,-0.146141,-0.377275,-0.185277,11.295307,2.384643,0.679496,0.858106,11.286035,0.404853,11.545770
2,0.368165,-0.207616,-0.165561,0.454376,0.162556,0.709293,3.968636,-2.902182,-2.103241,-0.018281,0.079766,-0.329582,-2.485754,6.100713,-0.485211,0.453938,0.857892,5.347556,0.339590,6.605536


In [23]:
time_sig_dic['exp02_user01'].shape

(19285, 20)

### 데이터 샘플링

```
2.56초를 기준으로 Raw 데이터 샘플링
50%는 오버랩 하여 샘플링을 수행
```

In [24]:
def Windowing_type(time_sig_dic,Labels_Data_Frame):
    columns=time_sig_dic['exp01_user01'].columns
    window_ID=0
    time_dictionary_window={}
    BA_array=np.array(Labels_Data_Frame)
    
    for line in tqdm.tqdm(BA_array):
        file_key= 'exp' + normalize2(int(line[0]))  +  '_user' + normalize2(int(line[1]))
        
        if line.shape[0] == 5 : # train label
          act_ID=line[2]
          start_point=line[3]
          end_point = line[4]
        else :                  # test label
          act_ID='None'
          start_point = line[2]
          end_point = line[3]
        
        for cursor in range(start_point,end_point-127,64):
            end_point=cursor+128
            data=np.array(time_sig_dic[file_key].iloc[cursor:end_point])
            window=pd.DataFrame(data=data,columns=columns)
            key='t_W'+normalize5(window_ID)+'_'+file_key+'_act'+normalize2(act_ID)
            time_dictionary_window[key]=window
            window_ID=window_ID+1
        
        # import pdb;pdb.set_trace()
    
    return time_dictionary_window 

In [25]:
train_time_dictionary_window  = Windowing_type(time_sig_dic,train_Labels_Data_Frame)
test_time_dictionary_window  = Windowing_type(time_sig_dic,test_Labels_Data_Frame)

100%|██████████| 258/258 [00:02<00:00, 107.99it/s]


In [26]:
train_window = train_time_dictionary_window[sorted(train_time_dictionary_window.keys())[0]]
train_window.head()

,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_gyro_X,t_body_gyro_Y,t_body_gyro_Z,t_body_gyro_jerk_X,t_body_gyro_jerk_Y,t_body_gyro_jerk_Z,t_body_acc_mag,t_grav_acc_mag,t_body_acc_jerk_mag,t_body_gyro_mag,t_body_gyro_jerk_mag
0,-0.003068,0.009275,-0.049713,1.025297,-0.135052,0.153468,-0.064668,0.067917,-0.010441,0.023431,0.089860,0.049693,-0.193282,-0.501977,0.204316,0.050664,1.045479,0.094359,0.105324,0.575399
1,-0.004362,0.010634,-0.049921,1.025123,-0.135542,0.152980,-0.044544,0.049300,0.188938,0.019565,0.079820,0.053780,0.630392,0.991992,-0.376632,0.051227,1.045300,0.200280,0.098216,1.234218
2,-0.005253,0.011620,-0.046143,1.024940,-0.136014,0.152417,0.021576,-0.060893,0.020793,0.032173,0.099660,0.046247,-0.058003,-0.047758,-0.215589,0.047872,1.045099,0.067866,0.114481,0.228307
3,-0.004821,0.010402,-0.045727,1.024747,-0.136467,0.151784,-0.178322,-0.007597,0.229162,0.031013,0.098705,0.041935,-0.260247,-0.011292,0.465103,0.047142,1.044877,0.290469,0.111638,0.533083
4,-0.008388,0.010250,-0.041144,1.024545,-0.136900,0.151080,0.168810,-0.075081,-0.268611,0.025808,0.098479,0.051237,0.188867,0.892604,-0.057455,0.043223,1.044634,0.326015,0.113971,0.914174


In [27]:
train_window.shape

(128, 20)

In [28]:
sorted(train_time_dictionary_window.keys())[:10]

['t_W00000_exp01_user01_act05',
 't_W00001_exp01_user01_act05',
 't_W00002_exp01_user01_act05',
 't_W00003_exp01_user01_act05',
 't_W00004_exp01_user01_act05',
 't_W00005_exp01_user01_act05',
 't_W00006_exp01_user01_act05',
 't_W00007_exp01_user01_act05',
 't_W00008_exp01_user01_act05',
 't_W00009_exp01_user01_act05']

In [29]:
sorted(train_time_dictionary_window.keys())[5000:5010]

['t_W05000_exp43_user21_act01',
 't_W05001_exp43_user21_act01',
 't_W05002_exp43_user21_act01',
 't_W05003_exp43_user21_act01',
 't_W05004_exp43_user21_act01',
 't_W05005_exp43_user21_act01',
 't_W05006_exp43_user21_act01',
 't_W05007_exp43_user21_act01',
 't_W05008_exp43_user21_act01',
 't_W05009_exp43_user21_act03']

In [30]:
len(train_time_dictionary_window.keys())

7283

In [31]:
print("시간 도메인 Train 데이터 수 : {}".format(len(train_time_dictionary_window)))
print("시간 도메인 Test 데이터 수 : {}".format(len(test_time_dictionary_window)))
print("윈도우 크기(2.56s => 128개) : {}".format(len(train_window)))

시간 도메인 Train 데이터 수 : 7283
시간 도메인 Test 데이터 수 : 3116
윈도우 크기(2.56s => 128개) : 128


### 3.FFT(Fast Fourier Transform)을 통한 시간 도메인-> 주파수 도메인 변환

In [32]:
from scipy import fftpack
from numpy.fft import *

def fast_fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    
    return amplitude_f_signal

def fast_fourier_transform(t_window):
    f_window=pd.DataFrame()
    for column in t_window.columns:
        if 'grav' not in column:
            t_signal=np.array(t_window[column])
            f_signal= np.apply_along_axis(fast_fourier_transform_one_signal,0,t_signal)
            f_window["f_"+column[2:]]=f_signal
    return f_window

In [33]:
train_frequent_dictionary_window = {'f'+key[1:] : train_t_df.pipe(fast_fourier_transform) for key, train_t_df in tqdm.tqdm(train_time_dictionary_window.items())}
test_frequent_dictionary_window = {'f'+key[1:] : test_t_df.pipe(fast_fourier_transform) for key, test_t_df in tqdm.tqdm(test_time_dictionary_window.items())}

100%|██████████| 3116/3116 [00:33<00:00, 94.20it/s]


In [34]:
train_window = train_frequent_dictionary_window[sorted(train_frequent_dictionary_window.keys())[0]]
train_window.head()

,f_body_acc_X,f_body_acc_Y,f_body_acc_Z,f_body_acc_jerk_X,f_body_acc_jerk_Y,f_body_acc_jerk_Z,f_body_gyro_X,f_body_gyro_Y,f_body_gyro_Z,f_body_gyro_jerk_X,f_body_gyro_jerk_Y,f_body_gyro_jerk_Z,f_body_acc_mag,f_body_acc_jerk_mag,f_body_gyro_mag,f_body_gyro_jerk_mag
0,0.257558,0.055128,0.568459,0.004474,1.018802,2.057622,1.302252,2.880254,0.066856,0.574643,8.573938,3.442790,3.790846,25.481237,14.048363,73.764900
1,0.324072,1.169116,2.325911,0.794786,1.852232,4.739415,2.270811,8.786654,3.040972,5.543606,13.054902,4.222888,0.110791,1.628419,2.657498,6.699360
2,0.045438,0.300832,0.599853,0.220647,0.463545,1.179457,0.332364,2.154464,0.845314,1.462829,2.404663,0.720631,0.721155,0.921396,1.619180,3.680039
3,0.026573,0.162318,0.341578,0.195753,0.523164,0.458591,0.396876,1.331866,0.486980,2.359180,1.583174,0.965404,0.326605,1.490716,0.289519,2.877012
4,0.040355,0.192653,0.218084,0.391329,0.872601,1.024741,0.484334,0.835291,0.432675,4.736053,0.525072,1.014540,0.252132,0.395907,0.434509,0.690402


In [35]:
print("주파수 도메인 Train 데이터 수 : {}".format(len(train_frequent_dictionary_window)))
print("주파수 도메인 Test 데이터 수 : {}".format(len(test_frequent_dictionary_window)))
print("피처의 갯수 : {}".format(len(train_window)))

주파수 도메인 Train 데이터 수 : 7283
주파수 도메인 Test 데이터 수 : 3116
피처의 갯수 : 128


In [36]:
len(train_frequent_dictionary_window.keys())

7283

In [37]:
train_frequent_dictionary_window['f_W00000_exp01_user01_act05']

,f_body_acc_X,f_body_acc_Y,f_body_acc_Z,f_body_acc_jerk_X,f_body_acc_jerk_Y,f_body_acc_jerk_Z,f_body_gyro_X,f_body_gyro_Y,f_body_gyro_Z,f_body_gyro_jerk_X,f_body_gyro_jerk_Y,f_body_gyro_jerk_Z,f_body_acc_mag,f_body_acc_jerk_mag,f_body_gyro_mag,f_body_gyro_jerk_mag
0,0.257558,0.055128,0.568459,0.004474,1.018802,2.057622,1.302252,2.880254,0.066856,0.574643,8.573938,3.442790,3.790846,25.481237,14.048363,73.764900
1,0.324072,1.169116,2.325911,0.794786,1.852232,4.739415,2.270811,8.786654,3.040972,5.543606,13.054902,4.222888,0.110791,1.628419,2.657498,6.699360
2,0.045438,0.300832,0.599853,0.220647,0.463545,1.179457,0.332364,2.154464,0.845314,1.462829,2.404663,0.720631,0.721155,0.921396,1.619180,3.680039
3,0.026573,0.162318,0.341578,0.195753,0.523164,0.458591,0.396876,1.331866,0.486980,2.359180,1.583174,0.965404,0.326605,1.490716,0.289519,2.877012
4,0.040355,0.192653,0.218084,0.391329,0.872601,1.024741,0.484334,0.835291,0.432675,4.736053,0.525072,1.014540,0.252132,0.395907,0.434509,0.690402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0.022292,0.063689,0.135585,0.268623,0.744189,0.455846,0.025241,0.587993,0.327638,0.491411,1.575543,0.613706,0.221429,0.841853,0.097779,1.083811
124,0.040355,0.192653,0.218084,0.391329,0.872601,1.024741,0.484334,0.835291,0.432675,4.736053,0.525072,1.014540,0.252132,0.395907,0.434509,0.690402
125,0.026573,0.162318,0.341578,0.195753,0.523164,0.458591,0.396876,1.331866,0.486980,2.359180,1.583174,0.965404,0.326605,1.490716,0.289519,2.877012
126,0.045438,0.300832,0.599853,0.220647,0.463545,1.179457,0.332364,2.154464,0.845314,1.462829,2.404663,0.720631,0.721155,0.921396,1.619180,3.680039


In [38]:
train_time_dictionary_window['t_W00000_exp01_user01_act05']

,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_gyro_X,t_body_gyro_Y,t_body_gyro_Z,t_body_gyro_jerk_X,t_body_gyro_jerk_Y,t_body_gyro_jerk_Z,t_body_acc_mag,t_grav_acc_mag,t_body_acc_jerk_mag,t_body_gyro_mag,t_body_gyro_jerk_mag
0,-0.003068,0.009275,-0.049713,1.025297,-0.135052,0.153468,-0.064668,0.067917,-0.010441,0.023431,0.089860,0.049693,-0.193282,-0.501977,0.204316,0.050664,1.045479,0.094359,0.105324,0.575399
1,-0.004362,0.010634,-0.049921,1.025123,-0.135542,0.152980,-0.044544,0.049300,0.188938,0.019565,0.079820,0.053780,0.630392,0.991992,-0.376632,0.051227,1.045300,0.200280,0.098216,1.234218
2,-0.005253,0.011620,-0.046143,1.024940,-0.136014,0.152417,0.021576,-0.060893,0.020793,0.032173,0.099660,0.046247,-0.058003,-0.047758,-0.215589,0.047872,1.045099,0.067866,0.114481,0.228307
3,-0.004821,0.010402,-0.045727,1.024747,-0.136467,0.151784,-0.178322,-0.007597,0.229162,0.031013,0.098705,0.041935,-0.260247,-0.011292,0.465103,0.047142,1.044877,0.290469,0.111638,0.533083
4,-0.008388,0.010250,-0.041144,1.024545,-0.136900,0.151080,0.168810,-0.075081,-0.268611,0.025808,0.098479,0.051237,0.188867,0.892604,-0.057455,0.043223,1.044634,0.326015,0.113971,0.914174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,-0.003679,-0.014323,0.000438,1.022534,-0.110934,0.100162,-0.019081,0.185830,-0.126946,0.056079,-0.094259,-0.033718,-0.336466,0.019835,0.337025,0.014795,1.033399,0.225859,0.114746,0.476643
124,-0.004061,-0.010606,-0.002101,1.022709,-0.111246,0.101207,0.194606,0.051416,-0.280300,0.049349,-0.093863,-0.026977,0.005888,0.436156,0.046692,0.011550,1.033708,0.345085,0.109423,0.438687
125,-0.000169,-0.009578,-0.007707,1.022878,-0.111574,0.102239,-0.031194,0.041146,-0.234494,0.049467,-0.085140,-0.026043,-0.318205,-0.379045,0.155929,0.012295,1.034012,0.240111,0.101853,0.518887
126,-0.000793,-0.008755,-0.012397,1.023040,-0.111917,0.103255,-0.037031,0.010671,0.064433,0.043103,-0.092721,-0.022925,-0.216222,0.653534,0.204675,0.015197,1.034311,0.075079,0.104788,0.718158


### 4. Feature Extract

In [39]:
train_time_dictionary_window['t_W00000_exp01_user01_act05'][train_time_dictionary_window['t_W00000_exp01_user01_act05'].columns[:15]]

,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_gyro_X,t_body_gyro_Y,t_body_gyro_Z,t_body_gyro_jerk_X,t_body_gyro_jerk_Y,t_body_gyro_jerk_Z
0,-0.003068,0.009275,-0.049713,1.025297,-0.135052,0.153468,-0.064668,0.067917,-0.010441,0.023431,0.089860,0.049693,-0.193282,-0.501977,0.204316
1,-0.004362,0.010634,-0.049921,1.025123,-0.135542,0.152980,-0.044544,0.049300,0.188938,0.019565,0.079820,0.053780,0.630392,0.991992,-0.376632
2,-0.005253,0.011620,-0.046143,1.024940,-0.136014,0.152417,0.021576,-0.060893,0.020793,0.032173,0.099660,0.046247,-0.058003,-0.047758,-0.215589
3,-0.004821,0.010402,-0.045727,1.024747,-0.136467,0.151784,-0.178322,-0.007597,0.229162,0.031013,0.098705,0.041935,-0.260247,-0.011292,0.465103
4,-0.008388,0.010250,-0.041144,1.024545,-0.136900,0.151080,0.168810,-0.075081,-0.268611,0.025808,0.098479,0.051237,0.188867,0.892604,-0.057455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,-0.003679,-0.014323,0.000438,1.022534,-0.110934,0.100162,-0.019081,0.185830,-0.126946,0.056079,-0.094259,-0.033718,-0.336466,0.019835,0.337025
124,-0.004061,-0.010606,-0.002101,1.022709,-0.111246,0.101207,0.194606,0.051416,-0.280300,0.049349,-0.093863,-0.026977,0.005888,0.436156,0.046692
125,-0.000169,-0.009578,-0.007707,1.022878,-0.111574,0.102239,-0.031194,0.041146,-0.234494,0.049467,-0.085140,-0.026043,-0.318205,-0.379045,0.155929
126,-0.000793,-0.008755,-0.012397,1.023040,-0.111917,0.103255,-0.037031,0.010671,0.064433,0.043103,-0.092721,-0.022925,-0.216222,0.653534,0.204675


In [40]:
cols = train_time_dictionary_window['t_W00000_exp01_user01_act05'].columns[:15]
df = train_time_dictionary_window['t_W00000_exp01_user01_act05'][cols]
df.head()

,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_gyro_X,t_body_gyro_Y,t_body_gyro_Z,t_body_gyro_jerk_X,t_body_gyro_jerk_Y,t_body_gyro_jerk_Z
0,-0.003068,0.009275,-0.049713,1.025297,-0.135052,0.153468,-0.064668,0.067917,-0.010441,0.023431,0.089860,0.049693,-0.193282,-0.501977,0.204316
1,-0.004362,0.010634,-0.049921,1.025123,-0.135542,0.152980,-0.044544,0.049300,0.188938,0.019565,0.079820,0.053780,0.630392,0.991992,-0.376632
2,-0.005253,0.011620,-0.046143,1.024940,-0.136014,0.152417,0.021576,-0.060893,0.020793,0.032173,0.099660,0.046247,-0.058003,-0.047758,-0.215589
3,-0.004821,0.010402,-0.045727,1.024747,-0.136467,0.151784,-0.178322,-0.007597,0.229162,0.031013,0.098705,0.041935,-0.260247,-0.011292,0.465103
4,-0.008388,0.010250,-0.041144,1.024545,-0.136900,0.151080,0.168810,-0.075081,-0.268611,0.025808,0.098479,0.051237,0.188867,0.892604,-0.057455


In [41]:
df[0:3]

,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_gyro_X,t_body_gyro_Y,t_body_gyro_Z,t_body_gyro_jerk_X,t_body_gyro_jerk_Y,t_body_gyro_jerk_Z
0,-0.003068,0.009275,-0.049713,1.025297,-0.135052,0.153468,-0.064668,0.067917,-0.010441,0.023431,0.08986,0.049693,-0.193282,-0.501977,0.204316
1,-0.004362,0.010634,-0.049921,1.025123,-0.135542,0.152980,-0.044544,0.049300,0.188938,0.019565,0.07982,0.053780,0.630392,0.991992,-0.376632
2,-0.005253,0.011620,-0.046143,1.024940,-0.136014,0.152417,0.021576,-0.060893,0.020793,0.032173,0.09966,0.046247,-0.058003,-0.047758,-0.215589


In [42]:
df[cols[0:3]].head()

,t_body_acc_X,t_body_acc_Y,t_body_acc_Z
0,-0.003068,0.009275,-0.049713
1,-0.004362,0.010634,-0.049921
2,-0.005253,0.011620,-0.046143
3,-0.004821,0.010402,-0.045727
4,-0.008388,0.010250,-0.041144


In [43]:
from Feature_engineering import freqs

freqs.shape

(128,)

In [44]:
# -------------------------------------
# [Empty Module #1] Feature Engineering
# -------------------------------------

# -------------------------------------
# Feature Engineering
# -------------------------------------
# 목적: 제공된 36개의 시퀀스 도메인 데이터를 기반으로 유의미한 피처를 추출한다.
# 입력인자: 시간(time) 도메인 Feature 20개 , 주파수(frequency) 도메인 Feature 16개
# 출력인자: 분류모델 학습을 위한 Feature
# -------------------------------------

# ------------------------------------------------------------
# 구현 가이드라인 - 논문에서 제안하는 Feature Engineering 방법
# ------------------------------------------------------------
#
# mean(): Mean value
# std(): Standard deviation
# mad(): Median absolute deviation 
# max(): Largest value in array
# min(): Smallest value in array
# sma(): Signal magnitude area
# energy(): Energy measure. Sum of the squares divided by the number of values. 
# iqr(): Interquartile range 
# entropy(): Signal entropy
# arCoeff(): Autorregresion coefficients with Burg order equal to 4
# correlation(): correlation coefficient between two signals
# maxInds(): index of the frequency component with largest magnitude
# meanFreq(): Weighted average of the frequency components to obtain a mean frequency
# skewness(): skewness of the frequency domain signal 
# kurtosis(): kurtosis of the frequency domain signal 
# bandsEnergy(): Energy of a frequency interval within the 64 bins of the FFT of each window.
# angle(): Angle between to vectors.

import sys

# Time domain Feature Extract function

from Feature_engineering import mean_axial,std_axial,mad_axial,max_axial,min_axial, t_sma_axial, t_energy_axial,IQR_axial,entropy_axial, t_arburg_axial, t_corr_axial
from Feature_engineering import mean_mag,std_mag,mad_mag,max_mag,min_mag,t_sma_mag,t_energy_mag,IQR_mag,entropy_mag,t_arburg_mag

# Frequency domain Feature Extract function
from Feature_engineering import f_sma_axial,f_energy_axial,f_max_Inds_axial,f_mean_Freq_axial,f_skewness_and_kurtosis_axial,f_all_bands_energy_axial
from Feature_engineering import f_sma_mag,f_energy_mag,f_max_Inds_mag,f_mean_Freq_mag,f_skewness_mag,f_kurtosis_mag

# Additional Feature Extract function
from Feature_engineering import angle_features

def feature_extractor(time_dictionary,freq_dictionary, condition='train') :
    
    
    if condition is 'train' :
        total_data = []
        total_label = []
    elif condition is 'test' :
        total_data = []
        
    for i in tqdm.tqdm(range(len(time_dictionary))) :
        
        time_key = sorted(time_dictionary.keys())[i]
        freq_key = sorted(freq_dictionary.keys())[i]
        
        time_window = time_dictionary[time_key]
        freq_window = freq_dictionary[freq_key]
        
        if condition is 'train' :
          window_user_id= int(time_key[-8:-6]) # extract the user id from window's key
          window_activity_id=int(time_key[-2:]) # extract the activity id from the windows key
        elif condition is 'test' :
          window_user_id= int(time_key[-10:-8]) # extract the user id from window's key
          window_activity_id= 0
        else :
            print("Error")
            sys.exit()
            break;
            
        ##################################################################################
        
        
        # Time domain - Feature extractor - Part 1. axial(X,Y,Z) Features 
        
        #[0,1,2] : 't_body_acc_X', 't_body_acc_Y', 't_body_acc_Z'
        #[3,4,5] : 't_grav_acc_X','t_grav_acc_Y', 't_grav_acc_Z'
        #[6,7,8] : 't_body_acc_jerk_X','t_body_acc_jerk_Y', 't_body_acc_jerk_Z'
        #[9,10,11] : 't_body_gyro_X','t_body_gyro_Y', 't_body_gyro_Z'
        #[12,13,14] : 't_body_gyro_jerk_X', 't_body_gyro_jerk_Y', 't_body_gyro_jerk_Z'
        
        axial_columns = time_window.columns[0:15]
        axial_df = time_window[axial_columns] # X,Y,Z
        
        time_axial_features = []
        
        for col in range(0,15,3) : 
            
            # ------------------------------------------------------------
            # 구현 가이드라인 
            # ------------------------------------------------------------
            # 아래 time_3axial_vector 나타난 Feature를 계산하여야 한다.
            # 각각을 계산하기위한 함수는 'Feature_engineering.py'에 내제되어 있다.
            # ------------------------------------------------------------
            
            # 40 value per each 3-axial signals
            cols = axial_columns[col:col+3]
            mean_vector = mean_axial(axial_df[cols])
            std_vector = std_axial(axial_df[cols])
            mad_vector = mad_axial(axial_df[cols])
            max_vector = max_axial(axial_df[cols])
            min_vector = min_axial(axial_df[cols])
            sma_value = t_sma_axial(axial_df[cols])
            energy_vector = t_energy_axial(axial_df[cols])
            IQR_vector = IQR_axial(axial_df[cols])
            entropy_vector = entropy_axial(axial_df[cols])
            AR_vector = t_arburg_axial(axial_df[cols])
            corr_vector = t_corr_axial(axial_df[cols])
            time_3axial_vector = mean_vector + std_vector + mad_vector + max_vector + min_vector + [sma_value] + energy_vector + IQR_vector + entropy_vector + AR_vector + corr_vector
            
            # append these features to the global list of features
            time_axial_features= time_axial_features+ time_3axial_vector
        
        ##################################################################################
        
        # Time domain - Feature extractor - Part 2. Magnitude Features 
        
        #[15]'t_body_acc_mag'
        #[16]'t_grav_acc_mag'
        #[17]'t_body_acc_jerk_mag'
        #[18]'t_body_gyro_mag'
        #[19]'t_body_gyro_jerk_mag'
        
        mag_columns = time_window.columns[15:]
        mag_df = time_window[mag_columns]
        
        time_mag_features = []
        
        for col in mag_columns :
            
            # ------------------------------------------------------------
            # 구현 가이드라인 
            # ------------------------------------------------------------
            # 아래 col_mag_values 나타난 Feature를 계산하여야 한다.
            # 각각을 계산하기위한 함수는 'Feature_engineering.py'에 내제되어 있다.
            # ------------------------------------------------------------
            
            # 13 value per each t_mag_column
            mean_value = mean_mag(mag_df[col])
            std_value = std_mag(mag_df[col])
            mad_value = mad_mag(mag_df[col])
            max_value = max_mag(mag_df[col])
            min_value = min_mag(mag_df[col])
            sma_value = t_sma_mag(mag_df[col])
            energy_value = t_energy_mag(mag_df[col])
            IQR_value = IQR_mag(mag_df[col])
            entropy_value = entropy_mag(mag_df[col])
            AR_vector = t_arburg_mag(mag_df[col])
            col_mag_values = [mean_value, std_value, mad_value, max_value, min_value, sma_value, 
                              energy_value,IQR_value, entropy_value]+ AR_vector

            # col_mag_values will be added to the global list
            time_mag_features= time_mag_features+ col_mag_values

        
        ##################################################################################
        
        # Frequency domain - Feature extractor - Part 1. axial(X,Y,Z) Features 
        
        #[0,1,2] : 'f_body_acc_X', 'f_body_acc_Y', 'f_body_acc_Z'
        #[3,4,5] : 'f_body_acc_jerk_X','f_body_acc_jerk_Y', 'f_body_acc_jerk_Z'
        #[6,7,8] : 'f_body_gyro_X','f_body_gyro_Y', 'f_body_gyro_Z'
        #[9,10,11] : 'f_body_gyro_jerk_X','f_body_gyro_jerk_Y', 'f_body_gyro_jerk_Z'
        
        axial_columns=freq_window.columns[0:12]
        axial_df=freq_window[axial_columns]
        freq_axial_features=[]
        
        for col in range(0,12,3) :         
            # ------------------------------------------------------------
            # 구현 가이드라인 
            # ------------------------------------------------------------
            # 아래 freq_3axial_features 나타난 Feature를 계산하여야 한다.
            # 각각을 계산하기위한 함수는 'Feature_engineering.py'에 내제되어 있다.
            # ------------------------------------------------------------
            
            cols = axial_columns[col:col+3]
            mean_vector = mean_axial(axial_df[cols])
            std_vector = std_axial(axial_df[cols])
            mad_vector = mad_axial(axial_df[cols])
            max_vector = max_axial(axial_df[cols])
            min_vector = min_axial(axial_df[cols])
            sma_value = f_sma_axial(axial_df[cols])
            energy_vector = f_energy_axial(axial_df[cols])
            IQR_vector = IQR_axial(axial_df[cols])
            entropy_vector = entropy_axial(axial_df[cols])
            max_inds_vector = f_max_Inds_axial(axial_df[cols])
            mean_Freq_vector = f_mean_Freq_axial(axial_df[cols])
            skewness_and_kurtosis_vector = f_skewness_and_kurtosis_axial(axial_df[cols])
            bands_energy_vector = f_all_bands_energy_axial(axial_df[cols])
            freq_3axial_features = mean_vector + std_vector + mad_vector + max_vector + min_vector + [sma_value] + energy_vector + IQR_vector + entropy_vector + max_inds_vector + mean_Freq_vector + skewness_and_kurtosis_vector + bands_energy_vector
            freq_axial_features = freq_axial_features+ freq_3axial_features
        
        ##################################################################################
        
        # Frequency domain - Feature extractor - Part 2. Magnitude Features
        
        #[12]'f_body_acc_mag'
        #[13]'f_body_acc_jerk_mag'
        #[14]'f_body_gyro_mag'
        #[15]'f_body_gyro_jerk_mag'
        
        mag_columns=freq_window.columns[12:]
        mag_df=freq_window[mag_columns]
        
        freq_mag_features = []
        
        for col in mag_columns:
            # ------------------------------------------------------------
            # 구현 가이드라인 
            # ------------------------------------------------------------
            # 아래 col_mag_values에 나타난 Feature를 계산하여야 한다.
            # 각각을 계산하기위한 함수는 'Feature_engineering.py'에 내제되어 있다.
            # ------------------------------------------------------------
            
            # 13 value per each t_mag_column
            mean_value = mean_mag(mag_df[col])
            std_value = std_mag(mag_df[col])
            mad_value = mad_mag(mag_df[col])
            max_value = max_mag(mag_df[col])
            min_value = min_mag(mag_df[col])
            sma_value = f_sma_mag(mag_df[col])
            energy_value = f_energy_mag(mag_df[col])
            IQR_value = IQR_mag(mag_df[col])
            entropy_value = entropy_mag(mag_df[col])
            max_Inds_value = f_max_Inds_mag(mag_df[col])
            mean_Freq_value = f_mean_Freq_mag(mag_df[col]) 
            skewness_value = f_skewness_mag(mag_df[col])
            kurtosis_value = f_kurtosis_mag(mag_df[col])
            col_mag_values = [mean_value, std_value, mad_value, max_value, 
                              min_value, sma_value, energy_value,IQR_value, 
                              entropy_value, max_Inds_value, mean_Freq_value,
                              skewness_value, kurtosis_value]
            
            freq_mag_features= freq_mag_features+ col_mag_values
        
        ##################################################################################
        
        # Time domain - Feature extractor - Part 3. Additional Features 
        
        additional_features = angle_features(time_window)
                
        ##################################################################################
        
        total_features = time_axial_features + time_mag_features + freq_axial_features + freq_mag_features + additional_features
        
        total_data.append(total_features)
        if condition is 'train' :
            total_label.append(window_activity_id)
    
    total_data = np.array(total_data)
    if condition is 'train' :
        total_label = np.array(total_label)
    
    if condition is 'train' :
        return total_data, total_label
    elif condition is 'test' :
        return total_data

In [45]:
train_data, train_label = feature_extractor(train_time_dictionary_window,train_frequent_dictionary_window,condition='train')
test_data = feature_extractor(test_time_dictionary_window,test_frequent_dictionary_window,condition='test')

100%|██████████| 3116/3116 [05:07<00:00, 10.15it/s]


In [46]:
train_data

array([[ 2.01216985e-03,  4.30685562e-04,  4.44108667e-03, ...,
         1.52887931e-01,  1.69216859e+00,  1.47828408e+00],
       [-7.13038377e-04, -3.09778156e-03,  8.22632253e-04, ...,
         1.49532349e-01,  1.68735175e+00,  1.47754839e+00],
       [-3.00873506e-04,  4.02494276e-03, -4.27990370e-03, ...,
         1.57004038e-01,  1.69615790e+00,  1.47676952e+00],
       ...,
       [-1.24931973e-02,  3.25562438e-03, -2.13712546e-02, ...,
         2.20594193e-01,  1.78853052e+00,  1.60564384e+00],
       [-2.72191104e-03, -3.54555307e-03,  1.95121765e-03, ...,
         2.15954645e-01,  1.78638786e+00,  1.58312071e+00],
       [ 3.50199340e-04, -1.91615643e-03, -9.07551388e-03, ...,
         2.14407561e-01,  1.78442170e+00,  1.58881593e+00]])

In [47]:
train_data.shape

(7283, 640)

In [48]:
train_label.shape

(7283,)

In [49]:
train_label.shape

(7283,)

In [50]:
train_label

array([5, 5, 5, ..., 2, 2, 2])

In [51]:
test_data.shape

(3116, 640)

## 데이터 정규화

In [52]:
# -------------------------------------
# [Empty Module #2] Data Normalization
# -------------------------------------

# -------------------------------------
# Data Normalization
# -------------------------------------
# 목적: 앞서 구축한 train,test 셋에 대한 Feature를 정규화한다.
# 입력인자: train 셋에서 추출된 Feature, test 셋에서 추출된 Feature
# 출력인자: 정규화된 Feature Vector
# -------------------------------------


from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()
mm_scaler.fit(train_data)

train_data_mm = mm_scaler.transform(train_data)
test_data_mm = mm_scaler.transform(test_data)

# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# sklearn에서 제공하는 MinMaxScaler를 사용해 데이터 정규화를 진행한다.
# (MinMaxScaler가 아닌 다른 정규화를 사용할 수 있다.)
# ------------------------------------------------------------


In [53]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

std_scaler = StandardScaler()
std_scaler.fit(train_data)

train_data_std = std_scaler.transform(train_data)
test_data_std = std_scaler.transform(test_data)

In [54]:
train_data[:5]

array([[ 2.01216985e-03,  4.30685562e-04,  4.44108667e-03, ...,
         1.52887931e-01,  1.69216859e+00,  1.47828408e+00],
       [-7.13038377e-04, -3.09778156e-03,  8.22632253e-04, ...,
         1.49532349e-01,  1.68735175e+00,  1.47754839e+00],
       [-3.00873506e-04,  4.02494276e-03, -4.27990370e-03, ...,
         1.57004038e-01,  1.69615790e+00,  1.47676952e+00],
       [ 8.27079255e-04, -2.79537207e-03,  4.50903420e-03, ...,
         1.48222155e-01,  1.69496503e+00,  1.49026932e+00],
       [-1.12919249e-03,  5.69339577e-04, -3.43719235e-03, ...,
         1.53360472e-01,  1.69760912e+00,  1.48501645e+00]])

In [55]:
train_data_mm[:5]

array([[0.52566317, 0.5215746 , 0.55636703, ..., 0.07640701, 0.58993889,
        0.47937374],
       [0.49672277, 0.47420922, 0.51764118, ..., 0.07470212, 0.58825426,
        0.47910075],
       [0.50109976, 0.56982313, 0.46303223, ..., 0.07849832, 0.5913341 ,
        0.47881174],
       [0.51307808, 0.4782687 , 0.55709422, ..., 0.07403644, 0.59091691,
        0.48382107],
       [0.49230341, 0.52343586, 0.47205119, ..., 0.0766471 , 0.59184165,
        0.4818719 ]])

In [56]:
train_data_std[:5]

array([[ 0.19893768,  0.09217307,  0.59357913, ..., -0.70614405,
         0.38806086,  0.01941485],
       [-0.0930112 , -0.40034668,  0.10411285, ..., -0.71287072,
         0.37714078,  0.01742579],
       [-0.04885638,  0.5938761 , -0.58610429, ..., -0.69789281,
         0.39710488,  0.01531997],
       [ 0.07198009, -0.35813496,  0.60277036, ..., -0.71549716,
         0.39440057,  0.05181906],
       [-0.13759338,  0.11152704, -0.47211121, ..., -0.70519678,
         0.4003949 ,  0.03761699]])

## 분류 모델 학습 및 평가

In [57]:
from sklearn.model_selection import train_test_split

def validate(model, X, y, metric, params: dict = None) -> dict:
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, stratify=y, random_state=42)
    
    if params:
        model.set_params(**params)
        
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    train_score = metric(y_train_pred, y_train)
    test_score = metric(y_val_pred, y_val)
    
    result = {'train': train_score, 
              'test': test_score,
              'params': params,
             'model': model}
    
    return result

In [58]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

### non-scaled data

In [59]:
validate(DecisionTreeClassifier(random_state=42), train_data, train_label, accuracy_score)

{'train': 1.0,
 'test': 0.9379461834157057,
 'params': None,
 'model': DecisionTreeClassifier(random_state=42)}

### scaled data

In [60]:
validate(DecisionTreeClassifier(random_state=42), train_data_mm, train_label, accuracy_score)

{'train': 1.0,
 'test': 0.9379461834157057,
 'params': None,
 'model': DecisionTreeClassifier(random_state=42)}

In [61]:
validate(DecisionTreeClassifier(random_state=42), train_data_std, train_label, accuracy_score)

{'train': 1.0,
 'test': 0.9379461834157057,
 'params': None,
 'model': DecisionTreeClassifier(random_state=42)}

In [62]:
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(train_data, train_label)

DecisionTreeClassifier(random_state=42)

In [63]:
dt_clf.get_depth()

22

In [64]:
dt_clf.get_n_leaves()

188

### feature importances

In [65]:
dt_clf.feature_importances_

array([6.17167215e-05, 0.00000000e+00, 3.27167590e-04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.02029883e-01, 0.00000000e+00,
       0.00000000e+00, 2.20391307e-04, 0.00000000e+00, 0.00000000e+00,
       5.48780853e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.03251947e-03, 0.00000000e+00,
       0.00000000e+00, 3.30944358e-04, 0.00000000e+00, 0.00000000e+00,
       1.64834332e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.23975221e-04, 0.00000000e+00, 7.54672372e-04,
       0.00000000e+00, 5.29081431e-03, 1.10455399e-03, 1.28081910e-03,
       0.00000000e+00, 1.81767643e-03, 1.56665550e-03, 0.00000000e+00,
       0.00000000e+00, 5.73017398e-04, 0.00000000e+00, 0.00000000e+00,
       5.98204976e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.29582759e-01, 4.68358191e-03, 3.96592751e-03, 4.16247302e-03,
      

In [66]:
np.sum(dt_clf.feature_importances_ > 0)

140

In [67]:
# feature가 너무 적어져 학습이 잘 이루어지지 않음
np.sum(dt_clf.feature_importances_ > 0.01)

9

In [68]:
np.sum(dt_clf.feature_importances_ > 0.005)

18

In [69]:
np.sum(dt_clf.feature_importances_ > 0.001)

60

In [70]:
np.sum(dt_clf.feature_importances_ > 0)

140

In [71]:
important_cols = dt_clf.feature_importances_ > 0.001

In [72]:
train_data[:,important_cols].shape

(7283, 60)

In [73]:
for label in range(1, 5 + 1):
    print(f"label {label} : {np.sum(train_label==label)}")

label 1 : 1215
label 2 : 1064
label 3 : 974
label 4 : 1260
label 5 : 1388


### 불필요한 feature 제거한 데이터 생성

In [74]:
imp_train_data = train_data[:, important_cols]
imp_train_data_mm = train_data_mm[:, important_cols]
imp_train_data_std = train_data_std[:, important_cols]
imp_test_data = test_data[:, important_cols]
imp_test_data_mm = test_data_mm[:, important_cols]

### 불필요한 feature 제거해서 실험

전체 학습 데이터로 학습 후 DecisionTree의 feature_importances_ 출력해봤을 때 대부분이 0인 것을 알 수 있었고, 이것이 overfitting으로 이어질 수 있다고 판단했다.

In [75]:
validate(DecisionTreeClassifier(random_state=42), train_data, train_label, accuracy_score)

{'train': 1.0,
 'test': 0.9379461834157057,
 'params': None,
 'model': DecisionTreeClassifier(random_state=42)}

In [76]:
result = validate(DecisionTreeClassifier(random_state=42, criterion="entropy"), train_data, train_label, accuracy_score)
result

{'train': 1.0,
 'test': 0.9450851180669961,
 'params': None,
 'model': DecisionTreeClassifier(criterion='entropy', random_state=42)}

In [77]:
result['model'].get_depth()

14

In [78]:
validate(DecisionTreeClassifier(random_state=42, criterion="entropy", max_depth=12, min_samples_split=10, min_samples_leaf=5), train_data_mm, train_label, accuracy_score)

{'train': 0.9864518491395093,
 'test': 0.9500274574409665,
 'params': None,
 'model': DecisionTreeClassifier(criterion='entropy', max_depth=12, min_samples_leaf=5,
                        min_samples_split=10, random_state=42)}

In [79]:
validate(DecisionTreeClassifier(random_state=42,
                                class_weight="balanced"),
         imp_train_data_mm,
         train_label,
         accuracy_score)

{'train': 1.0,
 'test': 0.9516749038989566,
 'params': None,
 'model': DecisionTreeClassifier(class_weight='balanced', random_state=42)}

In [80]:
validate(DecisionTreeClassifier(random_state=42,
                               max_depth=15,
                               min_samples_split=4,
                               min_samples_leaf=1,),
         imp_train_data_mm,
         train_label,
         accuracy_score)

{'train': 0.9945075064079092,
 'test': 0.9577155409115871,
 'params': None,
 'model': DecisionTreeClassifier(max_depth=15, min_samples_split=4, random_state=42)}

In [81]:
validate(RandomForestClassifier(random_state=42),
         train_data[:,dt_clf.feature_importances_ > 0.001],
         train_label, accuracy_score)

NameError: name 'RandomForestClassifier' is not defined

In [82]:
validate(RandomForestClassifier(random_state=42),
         train_data[:,dt_clf.feature_importances_ > 0],
         train_label, accuracy_score)

NameError: name 'RandomForestClassifier' is not defined

In [83]:
validate(RandomForestClassifier(random_state=42),
         train_data,
         train_label, accuracy_score)

NameError: name 'RandomForestClassifier' is not defined

In [84]:
validate(RandomForestClassifier(random_state=42),
         train_data[:,dt_clf.feature_importances_ > 0.002],
         train_label, accuracy_score)

NameError: name 'RandomForestClassifier' is not defined

### parameters tuning

In [85]:
results = []

for max_depth_ in range(15, 20):
    for min_samples_split_ in range(2, 5):
        for min_samples_leaf_ in range(1, 5):            
            params = {'max_depth': max_depth_,
                      'min_samples_split': min_samples_split_,
                      'min_samples_leaf': min_samples_leaf_}
            results.append(validate(DecisionTreeClassifier(random_state=42),
                                  train_data, train_label, accuracy_score, params))

In [86]:
sorted(results, key=lambda x: x['test'], reverse=True)[:10]

[{'train': 0.9891980959355547,
  'test': 0.9516749038989566,
  'params': {'max_depth': 17, 'min_samples_split': 2, 'min_samples_leaf': 3},
  'model': DecisionTreeClassifier(max_depth=17, min_samples_leaf=3, random_state=42)},
 {'train': 0.9891980959355547,
  'test': 0.9516749038989566,
  'params': {'max_depth': 17, 'min_samples_split': 3, 'min_samples_leaf': 3},
  'model': DecisionTreeClassifier(max_depth=17, min_samples_leaf=3, min_samples_split=3,
                         random_state=42)},
 {'train': 0.9891980959355547,
  'test': 0.9516749038989566,
  'params': {'max_depth': 17, 'min_samples_split': 4, 'min_samples_leaf': 3},
  'model': DecisionTreeClassifier(max_depth=17, min_samples_leaf=3, min_samples_split=4,
                         random_state=42)},
 {'train': 0.9891980959355547,
  'test': 0.9516749038989566,
  'params': {'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 3},
  'model': DecisionTreeClassifier(max_depth=18, min_samples_leaf=3, random_state=42)},
 {'t

In [87]:
results = []

for max_depth_ in range(15, 20):
    for min_samples_split_ in range(2, 5):
        for min_samples_leaf_ in range(1, 5):            
            params = {'max_depth': max_depth_,
                      'min_samples_split': min_samples_split_,
                      'min_samples_leaf': min_samples_leaf_}
            results.append(validate(DecisionTreeClassifier(random_state=42),
                                  train_data_mm, train_label, accuracy_score, params))
            print(f'{max_depth_}, {min_samples_split_}, {min_samples_leaf_}')

15, 2, 1
15, 2, 2
15, 2, 3
15, 2, 4
15, 3, 1
15, 3, 2
15, 3, 3
15, 3, 4
15, 4, 1
15, 4, 2
15, 4, 3
15, 4, 4
16, 2, 1
16, 2, 2
16, 2, 3
16, 2, 4
16, 3, 1
16, 3, 2
16, 3, 3
16, 3, 4
16, 4, 1
16, 4, 2
16, 4, 3
16, 4, 4
17, 2, 1
17, 2, 2
17, 2, 3
17, 2, 4
17, 3, 1
17, 3, 2
17, 3, 3
17, 3, 4
17, 4, 1
17, 4, 2
17, 4, 3
17, 4, 4
18, 2, 1
18, 2, 2
18, 2, 3
18, 2, 4
18, 3, 1
18, 3, 2
18, 3, 3
18, 3, 4
18, 4, 1
18, 4, 2
18, 4, 3
18, 4, 4
19, 2, 1
19, 2, 2
19, 2, 3
19, 2, 4
19, 3, 1
19, 3, 2
19, 3, 3
19, 3, 4
19, 4, 1
19, 4, 2
19, 4, 3
19, 4, 4


In [88]:
sorted(results, key=lambda x: x['test'], reverse=True)[:10]

[{'train': 0.9891980959355547,
  'test': 0.9516749038989566,
  'params': {'max_depth': 17, 'min_samples_split': 2, 'min_samples_leaf': 3},
  'model': DecisionTreeClassifier(max_depth=17, min_samples_leaf=3, random_state=42)},
 {'train': 0.9891980959355547,
  'test': 0.9516749038989566,
  'params': {'max_depth': 17, 'min_samples_split': 3, 'min_samples_leaf': 3},
  'model': DecisionTreeClassifier(max_depth=17, min_samples_leaf=3, min_samples_split=3,
                         random_state=42)},
 {'train': 0.9891980959355547,
  'test': 0.9516749038989566,
  'params': {'max_depth': 17, 'min_samples_split': 4, 'min_samples_leaf': 3},
  'model': DecisionTreeClassifier(max_depth=17, min_samples_leaf=3, min_samples_split=4,
                         random_state=42)},
 {'train': 0.9891980959355547,
  'test': 0.9516749038989566,
  'params': {'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 3},
  'model': DecisionTreeClassifier(max_depth=18, min_samples_leaf=3, random_state=42)},
 {'t

In [89]:
results = []

for max_depth_ in range(8, 14):
    for min_samples_split_ in range(2, 5):
        for min_samples_leaf_ in range(1, 5):            
            params = {'max_depth': max_depth_,
                      'min_samples_split': min_samples_split_,
                      'min_samples_leaf': min_samples_leaf_}
            result = validate(DecisionTreeClassifier(random_state=42, criterion="entropy"),
                                  train_data_mm, train_label, accuracy_score, params)
            results.append(result)
            print(f'{max_depth_}, {min_samples_split_}, {min_samples_leaf_}')

8, 2, 1
8, 2, 2
8, 2, 3
8, 2, 4
8, 3, 1
8, 3, 2
8, 3, 3
8, 3, 4
8, 4, 1
8, 4, 2
8, 4, 3
8, 4, 4
9, 2, 1
9, 2, 2
9, 2, 3
9, 2, 4
9, 3, 1
9, 3, 2
9, 3, 3
9, 3, 4
9, 4, 1
9, 4, 2
9, 4, 3
9, 4, 4
10, 2, 1
10, 2, 2
10, 2, 3
10, 2, 4
10, 3, 1
10, 3, 2
10, 3, 3
10, 3, 4
10, 4, 1
10, 4, 2
10, 4, 3
10, 4, 4
11, 2, 1
11, 2, 2
11, 2, 3
11, 2, 4
11, 3, 1
11, 3, 2
11, 3, 3
11, 3, 4
11, 4, 1
11, 4, 2
11, 4, 3
11, 4, 4
12, 2, 1
12, 2, 2
12, 2, 3
12, 2, 4
12, 3, 1
12, 3, 2
12, 3, 3
12, 3, 4
12, 4, 1
12, 4, 2
12, 4, 3
12, 4, 4
13, 2, 1
13, 2, 2
13, 2, 3
13, 2, 4
13, 3, 1
13, 3, 2
13, 3, 3
13, 3, 4
13, 4, 1
13, 4, 2
13, 4, 3
13, 4, 4


In [90]:
sorted(results, key=lambda x: x['test'], reverse=True)[:10]

[{'train': 0.9967045038447455,
  'test': 0.9511257550796266,
  'params': {'max_depth': 12, 'min_samples_split': 3, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(criterion='entropy', max_depth=12, min_samples_split=3,
                         random_state=42)},
 {'train': 0.9963383376052728,
  'test': 0.9505766062602965,
  'params': {'max_depth': 11, 'min_samples_split': 2, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(criterion='entropy', max_depth=11, random_state=42)},
 {'train': 0.9963383376052728,
  'test': 0.9500274574409665,
  'params': {'max_depth': 12, 'min_samples_split': 4, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(criterion='entropy', max_depth=12, min_samples_split=4,
                         random_state=42)},
 {'train': 0.9987184181618455,
  'test': 0.9500274574409665,
  'params': {'max_depth': 13, 'min_samples_split': 3, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(criterion='entropy', max_depth=13, min_samples_split=3

### 중요피처들로

#### non scaled data

In [91]:
results = []

for max_depth_ in range(13, 20):
    for min_samples_split_ in range(2, 5):
        for min_samples_leaf_ in range(1, 5):            
            params = {'max_depth': max_depth_,
                      'min_samples_split': min_samples_split_,
                      'min_samples_leaf': min_samples_leaf_}
            result = validate(DecisionTreeClassifier(random_state=42),
                                  imp_train_data, train_label, accuracy_score, params)
            results.append(result)

In [92]:
sorted(results, key=lambda x: x['test'], reverse=True)[:10]

[{'train': 0.9952398388868546,
  'test': 0.9577155409115871,
  'params': {'max_depth': 14, 'min_samples_split': 2, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(max_depth=14, random_state=42)},
 {'train': 0.9945075064079092,
  'test': 0.9577155409115871,
  'params': {'max_depth': 15, 'min_samples_split': 4, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(max_depth=15, min_samples_split=4, random_state=42)},
 {'train': 0.9981691688026364,
  'test': 0.957166392092257,
  'params': {'max_depth': 17, 'min_samples_split': 2, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(max_depth=17, random_state=42)},
 {'train': 0.9959721713658001,
  'test': 0.957166392092257,
  'params': {'max_depth': 17, 'min_samples_split': 4, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(max_depth=17, min_samples_split=4, random_state=42)},
 {'train': 0.9956060051263274,
  'test': 0.956068094453597,
  'params': {'max_depth': 15, 'min_samples_split': 3, 'min_samples_leaf': 1}

#### minmax scaled data

In [93]:
results = []

for max_depth_ in range(15, 20):
    for min_samples_split_ in range(2, 5):
        for min_samples_leaf_ in range(1, 5):            
            params = {'max_depth': max_depth_,
                      'min_samples_split': min_samples_split_,
                      'min_samples_leaf': min_samples_leaf_,}
            result = validate(DecisionTreeClassifier(random_state=42),
                                  imp_train_data_mm, train_label, accuracy_score, params)
            results.append(result)

In [94]:
sorted(results, key=lambda x: x['test'], reverse=True)[:10]

[{'train': 0.9945075064079092,
  'test': 0.9577155409115871,
  'params': {'max_depth': 15, 'min_samples_split': 4, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(max_depth=15, min_samples_split=4, random_state=42)},
 {'train': 0.9981691688026364,
  'test': 0.957166392092257,
  'params': {'max_depth': 17, 'min_samples_split': 2, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(max_depth=17, random_state=42)},
 {'train': 0.9959721713658001,
  'test': 0.957166392092257,
  'params': {'max_depth': 17, 'min_samples_split': 4, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(max_depth=17, min_samples_split=4, random_state=42)},
 {'train': 0.9956060051263274,
  'test': 0.956068094453597,
  'params': {'max_depth': 15, 'min_samples_split': 3, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(max_depth=15, min_samples_split=3, random_state=42)},
 {'train': 0.9985353350421091,
  'test': 0.956068094453597,
  'params': {'max_depth': 19, 'min_samples_split': 3, 'm

In [95]:
# entropy
results = []

for max_depth_ in range(8, 12):
    for min_samples_split_ in range(2, 5):
        for min_samples_leaf_ in range(1, 5):            
            params = {'max_depth': max_depth_,
                      'min_samples_split': min_samples_split_,
                      'min_samples_leaf': min_samples_leaf_,
                      'criterion': 'entropy'}
            result = validate(DecisionTreeClassifier(random_state=42),
                                  imp_train_data_mm, train_label, accuracy_score, params)
            results.append(result)

In [96]:
sorted(results, key=lambda x: x['test'], reverse=True)[:10]

[{'train': 0.9983522519223728,
  'test': 0.9538714991762768,
  'params': {'max_depth': 11,
   'min_samples_split': 4,
   'min_samples_leaf': 1,
   'criterion': 'entropy'},
  'model': DecisionTreeClassifier(criterion='entropy', max_depth=11, min_samples_split=4,
                         random_state=42)},
 {'train': 0.9992676675210546,
  'test': 0.9533223503569467,
  'params': {'max_depth': 11,
   'min_samples_split': 2,
   'min_samples_leaf': 1,
   'criterion': 'entropy'},
  'model': DecisionTreeClassifier(criterion='entropy', max_depth=11, random_state=42)},
 {'train': 0.9985353350421091,
  'test': 0.9533223503569467,
  'params': {'max_depth': 11,
   'min_samples_split': 3,
   'min_samples_leaf': 1,
   'criterion': 'entropy'},
  'model': DecisionTreeClassifier(criterion='entropy', max_depth=11, min_samples_split=3,
                         random_state=42)},
 {'train': 0.9950567557671183,
  'test': 0.9527732015376167,
  'params': {'max_depth': 10,
   'min_samples_split': 2,
   'min_sa

#### std scaled data

In [97]:
results = []

for max_depth_ in range(13, 20):
    for min_samples_split_ in range(2, 5):
        for min_samples_leaf_ in range(1, 5):            
            params = {'max_depth': max_depth_,
                      'min_samples_split': min_samples_split_,
                      'min_samples_leaf': min_samples_leaf_}
            result = validate(DecisionTreeClassifier(random_state=42),
                                  imp_train_data_std, train_label, accuracy_score, params)
            results.append(result)

In [98]:
sorted(results, key=lambda x: x['test'], reverse=True)[:10]

[{'train': 0.9945075064079092,
  'test': 0.9577155409115871,
  'params': {'max_depth': 15, 'min_samples_split': 4, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(max_depth=15, min_samples_split=4, random_state=42)},
 {'train': 0.9952398388868546,
  'test': 0.957166392092257,
  'params': {'max_depth': 14, 'min_samples_split': 2, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(max_depth=14, random_state=42)},
 {'train': 0.9981691688026364,
  'test': 0.957166392092257,
  'params': {'max_depth': 17, 'min_samples_split': 2, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(max_depth=17, random_state=42)},
 {'train': 0.9959721713658001,
  'test': 0.957166392092257,
  'params': {'max_depth': 17, 'min_samples_split': 4, 'min_samples_leaf': 1},
  'model': DecisionTreeClassifier(max_depth=17, min_samples_split=4, random_state=42)},
 {'train': 0.9956060051263274,
  'test': 0.956068094453597,
  'params': {'max_depth': 15, 'min_samples_split': 3, 'min_samples_leaf': 1},

In [99]:
from sklearn.ensemble import RandomForestClassifier

validate(RandomForestClassifier(random_state=42),
         train_data_mm,
         train_label,
         accuracy_score)

{'train': 1.0,
 'test': 0.9807797913234486,
 'params': None,
 'model': RandomForestClassifier(random_state=42)}

In [100]:
validate(RandomForestClassifier(random_state=42,
                                warm_start=True),
         imp_train_data_mm,
         train_label,
         accuracy_score)

{'train': 1.0,
 'test': 0.9895661724327293,
 'params': None,
 'model': RandomForestClassifier(random_state=42, warm_start=True)}

In [101]:
result = validate(RandomForestClassifier(),
                  imp_train_data_mm,
                  train_label,
                  accuracy_score)

In [102]:
result['model'].estimators_[0].get_depth()

16

In [103]:
results = []

for max_depth_ in range(15, 20):
    for min_samples_split_ in range(2, 5):
        for min_samples_leaf_ in range(1, 5):            
            params = {'max_depth': max_depth_,
                      'min_samples_split': min_samples_split_,
                      'min_samples_leaf': min_samples_leaf_}
            result = validate(RandomForestClassifier(random_state=42),
                                  imp_train_data_mm, train_label, accuracy_score, params)
            results.append(result)

In [104]:
sorted(results, key=lambda x: x['test'], reverse=True)[:10]

[{'train': 1.0,
  'test': 0.9923119165293794,
  'params': {'max_depth': 18, 'min_samples_split': 3, 'min_samples_leaf': 1},
  'model': RandomForestClassifier(max_depth=18, min_samples_split=3, random_state=42)},
 {'train': 1.0,
  'test': 0.9917627677100495,
  'params': {'max_depth': 19, 'min_samples_split': 3, 'min_samples_leaf': 1},
  'model': RandomForestClassifier(max_depth=19, min_samples_split=3, random_state=42)},
 {'train': 1.0,
  'test': 0.9912136188907194,
  'params': {'max_depth': 15, 'min_samples_split': 3, 'min_samples_leaf': 1},
  'model': RandomForestClassifier(max_depth=15, min_samples_split=3, random_state=42)},
 {'train': 1.0,
  'test': 0.9912136188907194,
  'params': {'max_depth': 17, 'min_samples_split': 3, 'min_samples_leaf': 1},
  'model': RandomForestClassifier(max_depth=17, min_samples_split=3, random_state=42)},
 {'train': 1.0,
  'test': 0.9906644700713894,
  'params': {'max_depth': 16, 'min_samples_split': 3, 'min_samples_leaf': 1},
  'model': RandomForestClass

In [105]:
# -------------------------------------
# [Empty Module #3] RandomForest를 이용한 분류
# -------------------------------------

# -------------------------------------
# RandomForest를 이용한 분류
# -------------------------------------
# 목적: 앞서 완성한 train/test Feature를 RandomForest를 이용해 분류한다.
# 추가 안내 : 리더보드의 베이스라인은 random_state를 0으로 하였다.
# 입력인자: Feature vector(train/test)
# 출력인자: 분류결과
# -------------------------------------

from sklearn.tree import DecisionTreeClassifier

submit_csv = pd.read_csv('/kaggle/input/2021-ml-p10/sample_submit.csv')
for result in sorted(results, key=lambda x: x['test'], reverse=True)[:10]:
    model = RandomForestClassifier(random_state=42)
    params = result['params']
    model.set_params(**params)
    model.fit(imp_train_data_mm, train_label)
    
    y_pred = model.predict(imp_test_data_mm)
    submit_csv['Label'] = y_pred.astype('int')
    submit_csv.to_csv(f'submission_rf_mm_{params}.csv', index=False)
    

# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# sklearn에서 제공하는 RandomForest를 사용해 분류를 진행한다.
# (RandomForest를가 아닌 다른 분류모델을 사용할 수 있다.)
# ------------------------------------------------------------

|'max_depth'|'min_samples_split'|'min_samples_leaf'|'warm_start'|score|
|---|---|---|---|---|
|17|2|2|F|0.93196|
|17|3|2|F|0.93196|
|17|4|2|F|0.93196|
|17|4|2|T|0.93196|

In [106]:
# submit_csv = pd.read_csv('/kaggle/input/2021-ml-p10/sample_submit.csv')
# submit_csv['Label'] = y_pred
# submit_csv['Label'] = submit_csv['Label'].astype("int")
# submit_csv.to_csv("./result-rf.csv", index=False)